In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from turbulence_generator import create_turbulence_df, merge_turbulence_and_noturbulence

In [41]:
t_max = 40  # s
f = 40  # Hz
v = 100  # m/s
h = 1000  # m
transition_duration = 3
turb_start = t_max / 2

plane_parameters = {"b": 13.16}

turb_data = pd.DataFrame(
    index=np.around(
        np.linspace(
            0,
            t_max - 1 / f,
            t_max * f,
        ),
        6,
    ),  # Rounding to the microsecond to avoid index problems related to rounding
    columns=["turbulence", "u", "v", "w", "p", "q", "r"],
    data=0.0,
)
no_turb_data = turb_data.copy(deep=True)

no_turb_df = create_turbulence_df(
    v,
    h,
    plane_parameters,
    turbulence_start_time=0,
    turbulence_duration=int(turb_start+transition_duration),
    sampling_frequency=f,
    is_turbulence=False,
)
no_turb_data.update(no_turb_df)
turb_df = create_turbulence_df(
    v,
    h,
    plane_parameters,
    turbulence_start_time=turb_start,
    turbulence_duration=int(t_max-turb_start),
    sampling_frequency=f,
    is_turbulence=True,
    turbulence_level=1
)
turb_data.update(turb_df)
res = merge_turbulence_and_noturbulence(turb_data, no_turb_data, [turb_start], [t_max-turb_start], transition_duration, f, transition_function="cosine")

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
res_turb = res[res["turbulence"] == 1]
res_no_turb = res[res["turbulence"] == 0]
ax.plot(res_no_turb.index, res_no_turb["w"], color="blue")
ax.plot(res_turb.index, res_turb["w"], color="red")
ax.set_xlabel("Time [s]")
ax.set_ylabel("w [m/s]")
ax.legend(["No turbulence", "Turbulence"])
ax.set_title("Example of vertical perturbation")
fig.tight_layout()

saving_fp = Path.cwd() / "turbulence_example_w.pdf"
fig.savefig(saving_fp)
plt.show()